# Lösung der Coding-Aufgabe

In diesem Jupyter Notebook versuche ich die gestellte [Coding Aufgabe](https://github.com/kvn23/aufgabe_t2021/blob/master/README.md) mit Python zu lösen.
<br /> Dabei nutze ich hauptsächlich mir schon vertraute Bibliotheken, insbesondere [Pandas](https://pandas.pydata.org/) und [SQLAlchemy](https://www.sqlalchemy.org/).

In [1]:
import os, json
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy as sa
import sqlalchemy_utils as sau

from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="lvwr73") # siehe https://geopy.readthedocs.io/en/stable/#nominatim

### 1 Einlesen der Daten

Öffnen und Konvertieren der SQLite Datenbank in ein Pandas DataFrame:

In [2]:
con = sqlite3.connect("sources/fahrten_db.sqlite")

datenbank_df = pd.read_sql_query("SELECT * from FAHRTEN", con).set_index('fahrt_id')

con.close()

Öffnen und Zusammenfügen der JSON Files in ein Pandas DataFrame:

In [3]:
path_to_json = 'sources/dta_rechnungen/'

# Initialisierung der Liste zur Zwischenspeicherung der einzelnen DFs je JSON file
dfs = []

# Zuerst wird jedes einzelne JSON file eingelesen, in ein DF konvertiert und auf der dfs Liste angehängt
for file_name in [file for file in os.listdir(path_to_json) if file.endswith('.json')]:
  with open(path_to_json + file_name) as json_file:
    data = pd.read_json(json_file)
    dfs.append(data)

# Konkatenieren der einzelnen DFs in ein gemeinsames DF mit fahrt_id als Zeilenindizes
rechnungen_df = pd.concat(dfs, ignore_index=True).set_index('fahrt_id')

Zusammenführen der beiden Dataframes:

In [4]:
# Left Join über Zeilen-Indizes (fahrt_id)
merged_df = datenbank_df.join(rechnungen_df, rsuffix="_right")

Überprüfung ob Person und Datum je Rechnung mit Daten in Datenbank übereinstimmen:

In [5]:
merged_df.iloc[:len(rechnungen_df)].person.equals(merged_df.iloc[:len(rechnungen_df)].person_right)

True

In [6]:
merged_df.iloc[:len(rechnungen_df)].datum_fahrt_right.equals(merged_df.iloc[:len(rechnungen_df)].datum_fahrt_right)

True

Entfernen der identischen Spalten:

In [7]:
merged_df = merged_df.drop(["person_right", "datum_fahrt_right"], axis=1)

### 2 Ermitteln der Geodaten und Distanzberechnung

Ermitteln der Geodaten via Geopy und Nominatim:

In [8]:
# Listen mit jeweils Start- und Zieladressen
starts = list(merged_df["start"])
ziele = list(merged_df["ziel"])

# Initialisierung der Listen, welche später Spalten im DF bilden
start_latitude = []
start_longitude = []
ziel_latitude = []
ziel_longitude = []

# Reihenweise Ermittlung der Geodaten zu den Startadressen
for row in starts:
    geoc = gc.geocode(row)
    if geoc is not None:
        start_latitude.append(geoc.latitude)
        start_longitude.append(geoc.longitude)
    else:
        start_latitude.append(np.nan)
        start_longitude.append(np.nan)

# Reihenweise Ermittlung der Geodaten zu den Zieladressen
for row in ziele:
    geoc = gc.geocode(row)
    if geoc is not None:
        ziel_latitude.append(geoc.latitude)
        ziel_longitude.append(geoc.longitude)
    else:
        ziel_latitude.append(np.nan)
        ziel_longitude.append(np.nan)

In [9]:
# Einfügen der Listen als Spalten in das DF
merged_df["start_latitude"] = start_latitude
merged_df["start_longitude"] = start_longitude
merged_df["ziel_latitude"] = ziel_latitude
merged_df["ziel_longitude"] = ziel_longitude

Berechnung der Distanzen mit der Haversine Formel, basierend auf https://www.tjansson.dk/2021/03/vectorized-gps-distance-speed-calculation-for-pandas/:

In [10]:
def haversine(lon1, lat1, lon2, lat2):                                                                                                                                   
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])                                                  
    a = np.sin((lat2 - lat1) / 2.0)**2 + (np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0)**2)                 
    distance = 6371 * 2 * np.arcsin(np.sqrt(a))                                                                         
    return distance

In [11]:
# Neue Spalte in DF für die Distanz einfügen und berechnen der Werte via Klasse haversine()
merged_df["distanz"] = haversine(merged_df["start_longitude"], merged_df["start_latitude"], merged_df["ziel_longitude"], merged_df["ziel_latitude"])

### 3 Kurze Zwischenanalyse der Daten

In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, bf0001 to bf0060
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           60 non-null     object 
 1   datum_fahrt      60 non-null     object 
 2   start            60 non-null     object 
 3   ziel             60 non-null     object 
 4   datum_rechnung   30 non-null     object 
 5   betrag_rechnung  30 non-null     object 
 6   start_latitude   56 non-null     float64
 7   start_longitude  56 non-null     float64
 8   ziel_latitude    60 non-null     float64
 9   ziel_longitude   60 non-null     float64
 10  distanz          56 non-null     float64
dtypes: float64(5), object(6)
memory usage: 7.7+ KB


Auffälligkeiten:
- Es fehlen die Rechnungsdaten in 30 Zeilen (da auch nur 30 JSON files / Rechnungen vorhanden waren)
- Zu vier der 60 Startadressen konnten keine Geodaten ermittelt werden, wodurch in diesen Zeilen keine Distanzen errechnet werden konnten.
- Alternativ könnte die Spalte "betrag_rechnung" von String zu Float konvertiert werden

In [13]:
# Anzeigen der Zeilen mit evtl. fehlerhaften Startadressen
faulty_rows = merged_df[merged_df["start_longitude"].isnull()].index
merged_df.loc[faulty_rows]

,person,datum_fahrt,start,ziel,datum_rechnung,betrag_rechnung,start_latitude,start_longitude,ziel_latitude,ziel_longitude,distanz
fahrt_id,,,,,,,,,,,
bf0003,O2577,2021-03-20,"Repgostraße 2, 50931 Köln","Bergstraße 6-12, 42105 Wuppertal",2021-04-13,94 EUR,NaN,NaN,51.259594,7.143416,NaN
bf0007,S1650,2021-03-31,"Vitalistraße 202, 50827 Köln","Ostmerheimer Str. 200, 51109 Köln",2021-04-18,412 EUR,NaN,NaN,50.939504,7.053428,NaN
bf0033,T6309,2021-05-08,"Repgostraße 2, 50931 Köln","Vogelsangstraße 106, 42109 Wuppertal",NaN,NaN,NaN,NaN,51.275106,7.141964,NaN
bf0050,W4353,2021-05-23,"Vitalistraße 202, 50827 Köln","Ostmerheimer Str. 200, 51109 Köln",NaN,NaN,NaN,NaN,50.939504,7.053428,NaN


Nach einer kurzen Google-Suche stellt sich heraus, dass die korrekten Schreibweisen der beiden Straßennamen "Repgowstraße" und "Vitalisstraße" lauten.

### 4 Korrigieren der Zeilen mit fehlerhaften Adressen

Korrigieren der Adressen:

In [14]:
# Dictionary mit Korrekturen
to_replace = {"Repgostraße": "Repgowstraße",
              "Vitalistraße" : "Vitalisstraße"}

# Strings ersetzen
merged_df["start"] = merged_df["start"].replace(to_replace, regex=True)

Neuberechnen der Geodaten und Distanz in den fehlerhaften Zeilen:

In [15]:
for row in faulty_rows:
    geoc = gc.geocode(merged_df.loc[row]["start"])
    merged_df.loc[row,("start_latitude")] = geoc.latitude
    merged_df.loc[row,("start_longitude")] = geoc.longitude
    merged_df.loc[row,("distanz")] = haversine(geoc.longitude, geoc.latitude, merged_df.loc[row].ziel_longitude, merged_df.loc[row].ziel_latitude)

Kurzer Blick auf die Zeilen, um Korrekturen zu bestätigen:

In [16]:
merged_df.loc[faulty_rows]

,person,datum_fahrt,start,ziel,datum_rechnung,betrag_rechnung,start_latitude,start_longitude,ziel_latitude,ziel_longitude,distanz
fahrt_id,,,,,,,,,,,
bf0003,O2577,2021-03-20,"Repgowstraße 2, 50931 Köln","Bergstraße 6-12, 42105 Wuppertal",2021-04-13,94 EUR,50.929627,6.926728,51.259594,7.143416,39.688565
bf0007,S1650,2021-03-31,"Vitalisstraße 202, 50827 Köln","Ostmerheimer Str. 200, 51109 Köln",2021-04-18,412 EUR,50.957331,6.897979,50.939504,7.053428,11.068928
bf0033,T6309,2021-05-08,"Repgowstraße 2, 50931 Köln","Vogelsangstraße 106, 42109 Wuppertal",NaN,NaN,50.929627,6.926728,51.275106,7.141964,41.250447
bf0050,W4353,2021-05-23,"Vitalisstraße 202, 50827 Köln","Ostmerheimer Str. 200, 51109 Köln",NaN,NaN,50.957331,6.897979,50.939504,7.053428,11.068928


Zum Schluss, Konvertierung der Rechnungsbeträge von String zu Float:

In [17]:
merged_df["betrag_rechnung"] = merged_df["betrag_rechnung"].replace(" EUR","", regex=True).astype(float)
merged_df["betrag_rechnung"].head(3)

fahrt_id
bf0001    285.0
bf0002    353.0
bf0003     94.0
Name: betrag_rechnung, dtype: float64

Letzter Blick auf den Datensatz:

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, bf0001 to bf0060
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           60 non-null     object 
 1   datum_fahrt      60 non-null     object 
 2   start            60 non-null     object 
 3   ziel             60 non-null     object 
 4   datum_rechnung   30 non-null     object 
 5   betrag_rechnung  30 non-null     float64
 6   start_latitude   60 non-null     float64
 7   start_longitude  60 non-null     float64
 8   ziel_latitude    60 non-null     float64
 9   ziel_longitude   60 non-null     float64
 10  distanz          60 non-null     float64
dtypes: float64(6), object(5)
memory usage: 7.7+ KB


### 5 Überführung in SQLite Datenbank

Erzeugen einer neuen Datenbank im Ordner "lvwr73_output": 

In [19]:
os.remove("lvwr73_output/fahrten_dwh.sqlite")# um alte Datei zu Demonstrationszwecken zu löschen
engine = sa.create_engine("sqlite:///lvwr73_output/fahrten_dwh.sqlite", echo=False)

Schreiben des Dataframes in die Datenbank als neues Table "FAHRTEN_ABRECHNUNG_RAW":

In [20]:
merged_df.to_sql("FAHRTEN_ABRECHNUNG_RAW", con=engine, if_exists="replace")

Erzeugen der View "FAHRTEN_ABRECHNUNG":

In [21]:
connection = engine.connect()
metadata = sa.MetaData()
fahrten = sa.Table('FAHRTEN_ABRECHNUNG_RAW', metadata,
        sa.Column('fahrt_id', sa.String, primary_key=True),
        sa.Column('person', sa.String),
        sa.Column('datum_fahrt', sa.String),
        sa.Column('datum_rechnung', sa.String),
        sa.Column('start', sa.String),
        sa.Column('ziel', sa.String),
        sa.Column('distanz', sa.Float),
        sa.Column('betrag_rechnung', sa.Float)
    )

selection = sa.select([fahrten]).where(fahrten.columns.datum_rechnung.isnot(None))

sau.create_view('FAHRTEN_ABRECHNUNG', selection, metadata)

metadata.create_all(engine)


In [22]:
connection.close()